# ARIMA + LSTM Architectures on residuals

    1) 'Vanilla' LSTM
    2) Stacked LSTM
    3) Bidirectional LSTM
    4) Encoder-Decoder LSTM-LSTM
    5) Encoder-Decoder CNN-LSTM


## Ideas
- Train the model on several stocks and try to predict one of them
- Cluster stocks by correlation - make a 2D projection in order to use in Tableau

In [ ]:
import pandas as pd
import yfinance as yf
import timeseries as ts

def get_data():
    sentic = pd.read_csv('sentic.csv', index_col=0, parse_dates=True)
    sentic.index.freq='D'
    sentic.to_excel('sentic.xlsx', sheet_name='Sentiment Analysis')
    
    btc = yf.download('BTC-USD', start='2020-02-14', end='2022-09-23', period='1d')[['Close']]
    btc.columns = ['btc']
    btc.index = sentic.index
    btc.to_excel('btc.xlsx', sheet_name='BTC Closing Price')

    #     sentic.index = pd.to_datetime(sentic.index).tz_localize('UTC')
    data = pd.concat([sentic,btc], axis=1)
    return data, sentic, btc

# data, sentic, btc = get_data()

sentiment = sentic['rate'].copy()

In [ ]:
def make_lags(df, n_lags=1, lead_time=1):
    """
    Compute lags of a pandas.DataFrame from lead_time to lead_time + n_lags. Alternatively, a list can be passed as n_lags.
    Returns a pd.DataFrame resulting from the concatenation of df's shifts.
    """

    if isinstance(n_lags,int):
        lag_list = range(lead_time, n_lags+lead_time)
    else:
        lag_list = n_lags
    
    lags=list()
    for i in lag_list:
        df_lag = df.shift(i)
        if i!=0:                
            df_lag.columns = [f'{col}_lag_{i}' for col in df.columns]
        lags.append(df_lag) 

    return pd.concat(lags, axis=1)



def add_dim(df, timesteps=5):
    """
    Transforms a pd.DataFrame into a 3D np.array with shape (n_samples, timesteps, n_features)
    """
    df = np.array(df)
    array_3d = df.reshape(df.shape[0],timesteps ,df.shape[1]//timesteps)
    return array_3d     



def prepare_data(df, target_name, n_lags, n_steps, lead_time, test_size, normalize=True, arima={order=(2,1,0),seasonal=(10,0,0,0)}):
    '''
    Prepare data for LSTM. 
    '''
    
    if arima:
        from statsmodels.tsa.arima_model import ARIMA
        arima = ARIMA(df[target_name], order=arima).fit()
        residuals = arima.residuals
        df.insert(df.columns.get_loc(target_name)+1, 'ARIMA_resid', residuals)
    
    if isinstance(n_steps,int):
        n_steps = range(1,n_steps+1)
    
    n_steps_reverted = [-x for x in list(n_steps)]

    X = make_lags(df, n_lags=n_lags, lead_time=lead_time).dropna()
    y = make_lags(df[[target_name]], n_lags=n_steps_reverted).dropna()
    
    last_row = X.iloc[-1:,:]
    
    X, y = X.align(y, join='inner', axis=0)
    
    from sklearn.model_selection import train_test_split    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=test_size, shuffle=False)


    
    if normalize:
        from sklearn.preprocessing import MinMaxScaler
        mms = MinMaxScaler().fit(X_train)
        X_train, X_val = mms.transform(X_train), mms.transform(X_val)    
        last_row = mms.transform(last_row)
        

        
        
    if isinstance(n_lags,int):
        timesteps = n_lags
    else:
        timesteps = len(n_lags)
    
    return add_dim(X_train, timesteps), add_dim(X_val, timesteps), y_train, y_val, add_dim(last_row, timesteps)



epochs, batch_size, verbose = 50, 72, 0

model_params={}

def fit_model(model, learning_rate=0.001, time_distributed=False, epochs=epochs, batch_size=batch_size, verbose=verbose, model_params=model_params):
    
    y_ind = y_val.index
    
    if time_distributed:
        y_train_0 = y_train.to_numpy().reshape((y_train.shape[0], y_train.shape[1],1))
        y_val_0 = y_val.to_numpy().reshape((y_val.shape[0], y_val.shape[1],1))        
    
    else:
        y_train_0 = y_train
        y_val_0 = y_val
    
    # fit network
    from keras.optimizers import Adam
    adam = Adam(learning_rate=learning_rate)
    model.compile(loss='mse', optimizer='adam')
    
    history = model.fit(X_train, y_train_0, 
                        epochs=epochs,
                        batch_size=batch_size, 
                        verbose=verbose,
                        **model_params, 
                        validation_data=(X_val, y_val_0), 
                        shuffle=False) 

    # make predictions and forecast
    if time_distributed:
        predictions = model.predict(X_val)[:,:,0]
        future_forecast = model.predict(last_row)[:,:,0]
    else:
        predictions = model.predict(X_val)
        future_forecast = model.predict(last_row)

    # preparing for plot
    yhat = pd.DataFrame(predictions, index=y_ind, columns=[f'pred_lag_{i}' for i in range(-n_steps,0)])
    yhat_shifted = pd.concat([yhat.iloc[:,i].shift(-n_steps+i) for i in range(len(yhat.columns))], axis=1)


    future_index = pd.date_range(start=y_val.index[-1]+pd.DateOffset(1+n_steps), periods=n_steps)
    future_forecast = pd.DataFrame(future_forecast[0], index=future_index)
    
    # saving forecast and model architecture
    name = model.name
    pd.concat([btc.squeeze(),future_forecast]).to_excel(name+'_forecast.xlsx', sheet_name=name+' Forecast')
    model.save(name+'model.h5')
    
    from keras.utils import plot_model
    img_file = name+'_arch.png'
    plot_model(model, to_file=img_file, show_shapes=True, show_layer_names=True)
    
    
    # calculate RMSE
    from sklearn.metrics import mean_squared_error, r2_score
    rmse = np.sqrt(mean_squared_error(y_val, yhat))
    
    import matplotlib.pyplot as plt
    
    fig, (ax1,ax2) = plt.subplots(2,1,figsize=(14,14))

    y_val.iloc[:,0].plot(ax=ax2,legend=True)
    yhat_shifted.plot(ax=ax2, alpha=.2)
    yhat_shifted.iloc[:,0].plot(ax=ax2, color='red')
    ax2.set_title('Prediction comparison')
    ax2.annotate(f'RMSE: {rmse:.5f} \n R2 score: {r2_score(yhat,y_val):.5f}', xy=(.68,.93),  xycoords='axes fraction')

    ax1.plot(history.history['loss'], label='train')
    ax1.plot(history.history['val_loss'], label='test')
    ax1.legend()

    plt.show()
    return model, future_forecast

def merge_forecasts():
    from glob import glob
    filenames = glob("*_forecast.xlsx")

    dfs = []

    for name in filenames: 
        dfs.append(pd.read_excel(name, index_col=0))
    
    df = pd.concat(dfs, axis=1)
    df.columns = [name.rstrip(r'_forecast.xlsx') for name in filenames]
    df.to_excel('forecast_all.xlsx', sheet_name='DL Forecast')
    return df, filenames

# Vanilla

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import numpy as np
from keras.callbacks import EarlyStopping


def create_output():
    output = Sequential()
    output.add(Dropout(.2, name='Dropout_1'))
    output.add(Dense(100, name='Dense_1'))
    output.add(Dropout(.2, name='Dropout_2'))
    output.add(Dense(n_steps, name='Dense_2'))
    return output




n_lags, n_steps, lead_time, test_size = 11, 11, 0, .2

epochs, batch_size, verbose = 100, 72, 0

model_params = {'callbacks':[EarlyStopping(
    monitor="val_loss",
    patience=20,
    mode="auto")]}



X_train, X_val, y_train, y_val, last_row = prepare_data(data, 'btc', n_lags, n_steps, lead_time, test_size)


vanilla = Sequential(name='Vanilla')
vanilla.add(LSTM(units=200, activation='relu', input_shape=(X_train.shape[1],X_train.shape[2]) ))
# vanilla.add(Dropout(.2, name='Dropout_1'))
# vanilla.add(Dense(100, name='Dense_1'))
# vanilla.add(Dropout(.2, name='Dropout_2'))
# vanilla.add(Dense(n_steps, name='Dense_2'))
vanilla.add(create_output())




fit_model(vanilla, learning_rate=0.001, time_distributed=False, epochs=epochs, batch_size=batch_size, verbose=verbose, model_params=model_params)
    

# Stacked

In [ ]:
stacked = Sequential(name = 'Stacked')
stacked.add(LSTM(units=100, activation='relu', input_shape=(X_train.shape[1],X_train.shape[2]), return_sequences=True ))
stacked.add(LSTM(units=200, activation='relu' ))
stacked.add(output)

In [ ]:
fit_model(stacked, learning_rate=0.001, time_distributed=False, epochs=epochs, batch_size=batch_size, verbose=verbose, model_params=model_params)

# Bidirectional

In [ ]:
from keras.layers import Bidirectional

bilstm = Sequential(name='Bidirectional')
bilstm.add(Bidirectional(LSTM(100, activation='relu'), input_shape=(X_train.shape[1], X_train.shape[2])))
bilstm.add(Sequential([
    Dropout(.2, name='Dropout_1'),
    Dense(100, name='Dense_1'),
    Dropout(.2, name='Dropout_2'),
    Dense(n_steps, name='Dense_2')
])


In [ ]:
fit_model(bilstm)

# Bidirectional - shorter train

In [ ]:
from keras.layers import Bidirectional



n_lags, n_steps, lead_time, test_size = 11, 11, 0, .2

epochs, batch_size, verbose = 100, 72, 0

model_params = {'callbacks':[EarlyStopping(
    monitor="val_loss",
    patience=20,
    mode="auto")]}


data1 = data.loc['2022-05-15':,:] #.drop(columns=['neg_median','last','median'])

X_train, X_val, y_train, y_val, last_row = prepare_data(data1, 'btc', n_lags, n_steps, lead_time, test_size)


bilstm_short = Sequential(name='Bidirectional_Short')
bilstm_short.add(Bidirectional(LSTM(100, activation='relu'), input_shape=(X_train.shape[1], X_train.shape[2])))
bilstm_short.add(Sequential([
    Dropout(.2, name='Dropout_1'),
    Dense(100, name='Dense_1'),
    Dropout(.2, name='Dropout_2'),
    Dense(n_steps, name='Dense_2')
]))

fit_model(bilstm_short)

# LSTM-LSTM

In [ ]:
from keras.layers import RepeatVector, TimeDistributed

# Encoder
lstmlstm = Sequential(name='LSTM-LSTM')
lstmlstm.add(LSTM(100, activation='relu',  input_shape=(X_train.shape[1], X_train.shape[2])))
lstmlstm.add(RepeatVector(n_steps))
lstmlstm.add(Dropout(.2))

# Decoder
lstmlstm.add(LSTM(200, activation='relu', return_sequences=True))
lstmlstm.add(TimeDistributed(Sequential([
    Dropout(.2, name='Dropout_1'),
    Dense(100, name='Dense_1'),
    Dropout(.2, name='Dropout_2'),
    Dense(n_steps, name='Dense_2')
])))


In [ ]:
fit_model(lstmlstm, time_distributed=True)

# LSTM-LSTM - short training set

In [ ]:
from keras.layers import RepeatVector, TimeDistributed

# Encoder
lstmlstm_short = Sequential(name='LSTM-LSTM')
lstmlstm_short.add(LSTM(100, activation='relu',  input_shape=(X_train.shape[1], X_train.shape[2])))
lstmlstm_short.add(RepeatVector(n_steps))
lstmlstm_short.add(Dropout(.2))

# Decoder
lstmlstm_short.add(LSTM(200, activation='relu', return_sequences=True))
lstmlstm_short.add(TimeDistributed(output))


In [ ]:
n_lags, n_steps, lead_time, test_size = 11, 11, 0, .2

epochs, batch_size, verbose = 100, 72, 0

model_params = {'callbacks':[EarlyStopping(
    monitor="val_loss",
    patience=20,
    mode="auto")]}


data1 = data.loc['2022-05-15':,:].drop(columns=['neg_median','last','median'])

X_train, X_val, y_train, y_val, last_row = prepare_data(data1, 'btc', n_lags, n_steps, lead_time, test_size)

fit_model(lstmlstm_short, time_distributed=True)

# CNN-LSTM

In [ ]:
from keras.layers import Flatten, Conv1D, MaxPooling1D

# Encoder
cnn_lstm = Sequential(name='CNN-LSTM')
cnn_lstm.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
cnn_lstm.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
cnn_lstm.add(MaxPooling1D(pool_size=2))
cnn_lstm.add(Flatten())
cnn_lstm.add(RepeatVector(n_steps))

# Decoder
cnn_lstm.add(LSTM(200, activation='relu', return_sequences=True))
cnn_lstm.add(TimeDistributed(output))


epochs, batch_size, verbose = 300, 32, 0


fit_model(cnn_lstm, epochs=epochs, batch_size=batch_size, time_distributed=True)

# CNN-Bidirectional

In [ ]:

from keras.layers import Flatten, Conv1D, MaxPooling1D

# Encoder
bi_lstm = Sequential(name='CNN-LSTM')
bi_lstm.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
bi_lstm.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
bi_lstm.add(MaxPooling1D(pool_size=2))
bi_lstm.add(Flatten())
bi_lstm.add(RepeatVector(n_steps))

# Decoder
bi_lstm.add(Bidirectional(LSTM(100, activation='relu', return_sequences=True), input_shape=(X_train.shape[1], X_train.shape[2])))
bi_lstm.add(LSTM(200, activation='relu', return_sequences=True))
bi_lstm.add(TimeDistributed(output))


epochs, batch_size, verbose = 300, 32, 0


fit_model(cnn_lstm, epochs=epochs, batch_size=batch_size, time_distributed=True)

# Attention and Transformer

In [ ]:
from keras import layers

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res



def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)


